In [1]:
!pip install kaggle

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
kaggle_dictionary = json.load(open("kaggle.json"))


MessageError: Error: credential propagation was unsuccessful

In [12]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [13]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [14]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 21.4MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.0MB/s]


In [15]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [16]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [17]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [18]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [19]:
data.shape

(50000, 2)

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [22]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-22-c1a779bd745a>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [23]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [24]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [25]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


data preprocssing

In [26]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [27]:
print(X_test)
print(X_train)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [28]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

BUILDING MODEL

In [29]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [32]:
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_batch_size=0.2)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 239s 369ms/step - accuracy: 0.7398 - loss: 0.5161
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 252s 359ms/step - accuracy: 0.8614 - loss: 0.3362
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 224s 359ms/step - accuracy: 0.8793 - loss: 0.2955
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 356ms/step - accuracy: 0.8997 - loss: 0.2516
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 357ms/step - accuracy: 0.9021 - loss: 0.2402


In [33]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"loss:{loss},accuracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 118ms/step - accuracy: 0.8879 - loss: 0.2723
loss:0.2688966393470764,accuracy:0.8889999985694885


BUILDING AN PREDICTIVE SYSTEM

In [35]:
def predict_sentiment(review):

  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [36]:
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
The sentiment of the review is: negative
